#Στειροπούλου Ευαγγελία 
###Α.Μ. 111520180016
##Εργασία 2η Αναγνώριση Προτύπων - Μηχανική Μάθηση



#Face recognition: 

In [11]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_lfw_people ##Labeled Faces in the Wild
from sklearn.decomposition import PCA
import glob
import re
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report

In [32]:
def loadImages(set_number):
    image_paths = glob.glob('/home/eva/Documents/machine_learning/project2/faces/person*_*.png')

    image_matrix = []
    labels = []
    for path in image_paths:
        filename = path.split('/')[-1]
        person_id = int(filename.split('_')[0][6:])
        image_number = int(filename.split('_')[1].split('.')[0])

        if ((set_number == 1 and 1 <= image_number <= 7) or
            (set_number == 2 and 8 <= image_number <= 19) or
            (set_number == 3 and 20 <= image_number <= 31) or
            (set_number == 4 and 32 <= image_number <= 45) or
            (set_number == 5 and 46 <= image_number <= 64)):

            image_array = np.array(plt.imread(path).flatten())
            image_matrix.append(image_array)
            labels.append(person_id)
    
    return np.array(image_matrix), labels

# Usage examples:
set_number = 1
image_matrix,labels = loadImages(set_number)
print(f"Set {set_number}: Image matrix shape: {image_matrix.shape}")

set_number = 2
image_matrix,labels = loadImages(set_number)
print(f"Set {set_number}: Image matrix shape: {image_matrix.shape}")

set_number = 3
image_matrix,labels = loadImages(set_number)
print(f"Set {set_number}: Image matrix shape: {image_matrix.shape}")

set_number = 4
image_matrix,labels = loadImages(set_number)
print(f"Set {set_number}: Image matrix shape: {image_matrix.shape}")

set_number = 5
image_matrix,labels = loadImages(set_number)
print(f"Set {set_number}: Image matrix shape: {image_matrix.shape}")



Set 1: Image matrix shape: (70, 2500)
Set 2: Image matrix shape: (120, 2500)
Set 3: Image matrix shape: (120, 2500)
Set 4: Image matrix shape: (140, 2500)
Set 5: Image matrix shape: (190, 2500)


In [ ]:

# def normalization(faces):
#     X = np.ndarray(shape=faces.shape)  # Normalized data matrix
#     # normalization ton prosopon afairontas mesi timi kai diairontas me typiki apoklisi
#     for i in range(faces.shape[0]):
#         X[i] = (faces[i] - np.mean(faces, axis=0)) / np.std(faces, axis=0)
#     return X

# loadImages("/home/eva/Documents/machine_learning/project2/faces/", "1")

In [ ]:

X_set1, y_set1 = loadImages("/home/eva/Documents/machine_learning/project2/faces", '01')
   
average_face = np.mean(X_set1, axis=0)

X_train_set = normalization(X_set1)

print(X_train_set.shape)

accuracy = [[] for _ in range(len([9, 30]))]

f, axes = plt.subplots(1, 5, figsize=(20, 5))

for d_idx, d in enumerate([9, 30]):
    pca = PCA(n_components=d, whiten=True).fit(X_train_set)
    X_train_pca = pca.transform(X_train_set)
    classifier = KNeighborsClassifier(n_neighbors=1).fit(X_train_pca, y_set1)

    for i in range(1, 6):
        X_set, y_set = loadImages("/home/eva/Documents/machine_learning/project2/faces", + str(i))
        X_test_set = normalization(X_set)
        X_test_pca = pca.transform(X_test_set)
        y_pred = classifier.predict(X_test_pca)
        accuracy[d_idx].append(int(accuracy_score(y_set, y_pred) * 100))
        print(accuracy_score(y_set, y_pred))

#         names =[1,2,3,4,5,6,7,8,9,10]


#         # ektipono ektos apo to Score kai to confusion matrix
#         cfmatr = confusion_matrix(y_set, y_pred)
#         cmd = ConfusionMatrixDisplay(cfmatr, display_labels=names)
#         cmd.plot(ax=axes[i-1])
#         cmd.ax_.set_title("Set_"+str(i)+ " Score: "+str(int(accuracy_score(y_set, y_pred)*100))+"%")
#         cmd.im_.colorbar.remove()
#         cmd.ax_.set_xlabel('')

#   # gia to plotting ton confusion matrixes
#     plt.subplots_adjust(wspace=0.1, hspace=0.1)
#     plt.tight_layout()
#     plt.show()

##I: Συνάρτηση loadImages:

Όπως αναφέρεται στην εκφώνηση, η προεπεξεργασία της εικόνας είναι απαραραίτητη, και γίνεται με την αφαίρεση της μέσης τιμής της και στην συνέχεια με την διαίρεση με την τυπική απόκλιση των τιμών της.

In [19]:

def normalization(faces):
    X = np.ndarray(shape=faces.shape) #Normalized data matrix
    for i in range(faces.shape[0]):
        X[i] = (faces[i] - np.mean(faces))/np.std(faces) #Mean normalization of the data

    return X
